In [4]:
import sys
libpath = "/home/pablo/tesis/tsm-pipeline/code"
if libpath not in sys.path:
    sys.path.append(libpath)
    
%load_ext autoreload
%autoreload 2

import docker
docker_client = docker.from_env()

# Configura todo lo necesario en el ambiente
from scripts.xxx import run_tsm, ExperimentSettings, read_dbs_dataset
from scripts.evaluate import evaluate, EvaluationSettings
from os.path import join as path_join

defaults = {
    "bash_config_path": "/home/pablo/tesis/tsm-pipeline/code/scripts/config.sh",
}
defaults_evalute = {
    "cache_root":"/home/pablo/dbcache",    
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# el dataset de dbs de abajo son:
# - 100 random elejidas de
# - el 60% percentil superior de todas las xss, es decir, al menos 48% loc de js
xss_dbs = read_dbs_dataset("/home/pablo/tesis/tsm-pipeline/experiments/tesis/xss_random_100.txt")

In [12]:
from sklearn.model_selection import train_test_split
from numpy.random import RandomState

# Using a fixed random state to have reproducible results
rand = RandomState(seed=5)

train, test = train_test_split(xss_dbs, train_size=.5, shuffle=True, random_state=rand)

In [13]:
train[0:10]

['StephenGrider/ReactSSRCasts/99f281431594d8bb925331c5bb44793020fd8ba1',
 'GeoNode/geonode-contribs/45c15eb796b931a7164c07e89ae108b61558eb52',
 'danielstern/express-react-fullstack/865504ee0d0188f10d162a06d1311d9115a68cfe',
 'Polychart/polychart2/4ac837ad4ee007aa291fb18a9df9eae810191954',
 'GoogleChrome/chrome-app-samples/dc5d34f0f74bfd73a7dfa394bedbd69e123a5807',
 'Nicow1991/myIonicApp/a77364d438932c53ef3dfe871078f71eb07f425f',
 'davideast/firebase-ionic-login-sample/dc68eb5a1ca1620abe57f7e886fa68163346148a',
 'Sean12697/SlackChat/a123ba5d19af94ac222250ef1001d312b700224f',
 'KayFelicities/ExperienceBase/866ad2c78f0acea1169126843ed1051c5f424fa3',
 'AnonymousFunction/anonymousfunction.github.com/dc62be0bd555502d9af4d98791a49e795eeb11dc']

In [14]:
len(train)

50

Calculo un split de 50/50 con tainted path top 100

In [15]:
results_dir = "/home/pablo/results/seldon_xss_1"

In [5]:
results_dir = '/home/pablo/results/seldonstar_nosql_1'

In [16]:
# training
train_settings = ExperimentSettings(
    name="seldon_xss_1",
    query_type="xss",
    project_list=train,
    results_dir=results_dir,
    **defaults
)

run_tsm(docker_client, train_settings, block=True)

2023-05-02 20:21:25,966 - INFO - creating /home/pablo/results/seldon_xss_1 dir. Omitting error if already existing
2023-05-02 20:21:25,967 - INFO - creating /tmp/log dir. Omitting error if already existing
2023-05-02 20:21:26,361 - INFO - running at container 0f3d2854a93969b41b48c1f71db2db16ec949669cbb95533d5e4b025192529f0. Use `docker logs 0f3d2854a93969b41b48c1f71db2db16ec949669cbb95533d5e4b025192529f0 --tail 10 --follow` to follow progress
2023-05-02 22:54:40,049 - INFO - running combine scores
g: /home/pablo/results/seldon_xss_1/*/DomBasedXssWorse-*/reprScores.txt


In [17]:
!cat /home/pablo/results/seldon_xss_1/averaged-results.csv|wc -l

196


In [19]:
test[0:10]

['blup/bootstrap.stylus/a474c9e354ed284921b060fd8ee2489a9c71c96d',
 'd0nd3r3k/SimpleChat/17478868bc877c8507df83cdccf855a70f9f7c18',
 'JetBrains/svg-mixer/1b4b86f813fbabda1d40c2166c06095ab0a26f5b',
 'ccoenraets/directory-angular-ionic/fe78d4605b344ab868e5097a10323c8a2c818068',
 'cubicleDowns/ng-three-viewer/9c2717f6a4dabc6326290308b691af85da01629a',
 'amymok/dek-EU/b5dc79958b2f76c876d87e0dba2a7b093043b72b',
 'CyberSaxosTiGER/stream/3ac3836167a3f48c1c3f64ec50821f379bec1603',
 'aosabook/500lines/fba689d101eb5600f5c8f4d7fd79912498e950e2',
 'BrowserSync/browser-sync/6ffc212cdb50da972e4ed434244a23a6bee56e50',
 'RSO/slidetator/1d619b6b454fe6cb08681a725eea8ea6fe41f040']

In [22]:
# evaluation
external_predicate_file = path_join(results_dir, 'averaged-results.csv')
worse_settings = EvaluationSettings(
    search_path="/home/pablo/tesis/tsm-pipeline/lib-worse/codeql",
    cli_version="2.5.2",
    query_file="/home/pablo/tesis/tsm-pipeline/tsm-atm-pipeline/src/tsm/evaluation/XssWorseTSM.ql",
    external_predicate_file=external_predicate_file,
    **defaults_evalute,
)

print("Evaluating worse with %d dbs" % (len(test)))
evaluate(
    settings=worse_settings,
    output_dir=path_join(results_dir, 'worse'),
    dbs=test,
)

# v0_settings = EvaluationSettings(
#     search_path="/home/pablo/codeqlv0/javascript/ql/lib",
#     cli_version="2.10.5",
#     query_file="/home/pablo/tesis/tsm-pipeline/tsm-evaluation/tsm-evaluation/src/XssEvaluation.ql",
#     **defaults_evalute,
# )

# print("Evaluating v0 with %d dbs" % (len(test)))
# evaluate(
#     settings=v0_settings,
#     output_dir=path_join(results_dir, 'v0'),
#     dbs=test,
# )

Evaluating worse with 50 dbs


In [23]:
from scripts.calculate_scores import calculate_scores

calculate_scores(
    results_folder=results_dir,
    cleanup_base_dir="/tmp",
)

(0.0, 0.0, 0.0)